In [ ]:
%matplotlib inline
import os
from osgeo import gdal
import rasterio
import geopandas as gpd
import rasterio.mask
from rasterio.windows import Window
import sys
from shapely.geometry import mapping
sys.path.append(r'E:/gitlab/')
from codigos import Generar_txt
###path de yolo dentro de computadora
os.chdir(r'/yolov7')
from detect_Alberto_v4 import *
from scipy.ndimage import rotate as rotate_image
from shapely import geometry
import time
import datetime
from shapely.ops import cascaded_union

In [ ]:
raster=r".tif"

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jun 21 11:48:52 2023

@author: Alberto
"""
import tensorflow as tf
from tensorflow.keras import *
from tensorflow.keras.layers import *
import os

IMG_WIDTH=256
IMG_HEIGHT=IMG_WIDTH
CKPATH=r"D:\checkpoint"
PATH=r"\ortofoto"


def downsample(filters, apply_batchnorm=True):
  result=Sequential()
  initializer=tf.random_normal_initializer(0,0.02)
  #capa convolucional
  result.add(Conv2D(filters,
                    kernel_size=4,
                    strides=2,
                    padding="same",
                    kernel_initializer=initializer,
                    use_bias=not apply_batchnorm))
  if apply_batchnorm:
    #capa BatchNorm
    result.add(BatchNormalization())
  #capa de activacion
  result.add(LeakyReLU())
  return result

def upsample(filters, apply_dropout=False):
  result=Sequential()
  initializer=tf.random_normal_initializer(0,0.02)
  #capa convolucional
  result.add(Conv2DTranspose(filters,
                             kernel_size=4,
                             strides=2,
                             padding="same",
                             kernel_initializer=initializer,
                             use_bias=False))
  #capa BatchNorm
  result.add(BatchNormalization())
  if apply_dropout:
    #capa de Dropout
    result.add(Dropout(0.1))
  #capa de activacion
  result.add(LeakyReLU())##
  return result

def Generator():
    inputs=tf.keras.layers.Input(shape=[IMG_WIDTH,IMG_WIDTH,3])
    down_stack=[downsample(64,apply_batchnorm=True),
              downsample(128),
              downsample(256),
              downsample(512),
              downsample(512),
              downsample(512)]

    up_stack=[upsample(512,apply_dropout=True),
            upsample(512),
            upsample(256),
            upsample(128),
            upsample(64)]
    initializer=tf.random_normal_initializer(0,0.02)
    last=Conv2DTranspose(filters=3,
                       kernel_size=4,
                       strides=2,
                       padding="same",
                       kernel_initializer=initializer,
                       activation="tanh")  
    x=inputs
    s=[]
    concat=Concatenate()
    for down in down_stack:
        x=down(x)
        s.append(x)
    s=reversed(s[:-1]) 
    for up,sk in zip(up_stack,s):
        x=up(x)
        x=concat([x,sk])
    last=last(x)
    return Model(inputs=inputs,outputs=last)

generator=Generator()
generator.summary()

def Discriminator():
  ini=Input(shape=[None,None,3],name="input_img")
  gen=Input(shape=[None,None,3],name="gener_img")
  con=concatenate([ini,gen])
  initializer=tf.random_normal_initializer(0,0.02)
  down1=downsample(64,apply_batchnorm=False)(con)
  down2=downsample(128)(down1)
  down3=downsample(256)(down2)
  down4=downsample(512)(down3)
  last=tf.keras.layers.Conv2D(filters=1,
                              kernel_size=4,
                              strides=1,
                              kernel_initializer=initializer,
                              padding="same")(down4)
  return tf.keras.Model(inputs=[ini,gen],outputs=last)

discriminator=Discriminator()


generator_optimizer = tf.keras.optimizers.Adam(2e-4,beta_1=0.5)
discriminator_optimizer=tf.keras.optimizers.Adam(2e-4,beta_1=0.5)
 
checkpoint_prefix=os.path.join(CKPATH,"ortofoto_2")
checkpoint=tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                               discriminator_optimizer=discriminator_optimizer,
                              generator=generator,
                               discriminator=discriminator)
checkpoint.restore(tf.train.latest_checkpoint(CKPATH)).expect_partial()



In [ ]:
def pix2pix_gen(image):
    image=cv2.resize(image,(256,256))
    low=np.array([  0,   0, 140], dtype="uint8")
    high=np.array([179, 255, 255], dtype="uint8")
    hsv=cv2.cvtColor(np.array(image),cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv,low,high)
    im=image.copy()
    if np.sum(image[:,:,0])<10:
        y=[np.zeros_like(im)]
    else:
        image= image/127.5-1
        x=np.expand_dims(image, axis=0)
        y=generator(x, training=True)
    y=np.array(y[0]*255,dtype="uint8")#normalizar_array(y[0])
    y=np.array(y*np.abs(np.array(np.stack([mask,mask,mask],axis=-1)/255,dtype=int)),dtype="uint8")
    y=cv2.resize(y,(dim,dim))
    return y

In [ ]:
import time
metros=60
shape="none"
# funcion=sum
alto,ancho,dim,crs,H,W,minx,maxx,miny,maxy=Parametro_raster(raster,metros=metros)
# def generar_mosaico(raster,metros=80,shape="none",funcion=sum):
# alto,ancho,dim,crs,H,W,minx,maxx,miny,maxy=Parametro_raster(raster,metros=metros)

src=rasterio.open(raster)
with tqdm.tqdm(total=alto*ancho+1) as pbar:
    for j in range(ancho):#ancho
        for i in range(alto):#alto
            pbar.update(1)
            generar=0
            label=raster.replace('\\','/').split('/')[-1][:-4]+'_'
            nameimg=label.lower()+str(i)+'_'+str(j)
            cuadro=[]
            for k in range(2):
                for l in range(2):
                    cuadro.append((minx+(maxx-minx)/ancho*(j+k),maxy-(maxy-miny)/alto*(i+l)))
            cuadro=[cuadro[0],cuadro[1],cuadro[3],cuadro[2],cuadro[0]]
            cvees=[]
            for punto in cuadro:
                x=float(punto[0])
                y=float(punto[1])
                if type(shape)!=str:
                    if shape.contains(Point(x,y)):
                        generar=1
                else:
                    generar=1
            if generar==1:
                shapes=[{'type':'Polygon','coordinates':[cuadro]}]
                vector=[]
                left,bottom=np.min(cuadro,axis=0)
                rigth,top=np.max(cuadro,axis=0)
                left,bottom,rigth,top
                window=src.window(left,bottom,rigth,top)
                image =src.read([3,2,1],window=window)
                image=np.moveaxis(image,0,-1)
                y=pix2pix_gen(image)
            else:
                y=np.zeros((dim,dim,3))
            if (i==0):
                R=y
            else:
                R=np.concatenate((R,y), axis=0)
        if (j==0):
            S=R
        else:
            S=np.concatenate((S,R), axis=1)

#                         v.append(image)


In [ ]:
matriz=np.max(S,axis=-1)
# src=rasterio.open(raster)
transform=src.meta.get("transform")
out_meta=({"driver": "GTiff",
            "height": matriz.shape[0],
            "width": matriz.shape[1],
            "count":1,
            "dtype":np.float32,
            "crs":crs,
        
          'transform':transform})
with rasterio.open(r"E:\rastermunicpial\.tif",'w', **out_meta) as dest_file:
    dest_file.write(matriz,1)